## 1. Re-Calculate analysis period

##### Motivation:
- Among CI projects we found 76 without bugs in the selected period, however  with bugs in other periods.
- Several projects adopt CI service, but start doing automated builds and monitoring coverage only some time after. Therefore we can supose that indeed such projects employ CI on that date.

##### Procedure:
- We now consider the date of first coverage record for projects using Travis CI. This is due our criteria in consider CI projects those having automated builds and coverage metric.
- Columns were added to Projects table:
  - cov_init
  - cov_init_days
  - ci_service_init
  - ci_service_init_days
- Redifine analysis_point_days for projects using CI Service
- Get the new median days for CI projects (having cov_init not null) and set it on NO CI projects
- Redifine analysis_init and analysis_finish for all projects

##### Reminds:
- Re-execute merge_conflict process

##### 1.1 Setting Cov_init and Cov_init_days

- Get first coverage record to set cov_init
- Diff between days (repo_creation and cov_init) to set cov_init_days

In [ ]:
repos = getProjectsToRecalculate()

i=1
j=0
for repo in repos: 
    project = repo[0]
    repo_creation = repo[1]
    print('Processing Dates: {}\t\t {}\{}  -  {}'.format(project,i,len(repos),datetime.now().strftime("%d/%m/%y %H:%M:%S")))
    i+=1
    
    cov_init = getFirstCov(project)[0]
    build_init = getFirstBuild(project)[0]
    
    if cov_init is not None and build_init is not None and cov_init < build_init:
        cov_init = build_init
    
    print(cov_init)
    
    if cov_init is not None:
        days = abs(cov_init.replace(tzinfo=None) - repo_creation.replace(tzinfo=None)).days
        setCovInit(project,cov_init,days)
    

##### 1.2 Redifining analysis_point and analysis_point_days for projects using CI Service

- We ran the SQL:
   - update projects set analysis_point_days = cov_init_days, analysis_point = cov_init where ci_service like 'Travis CI';

##### 1.3. Get the new median days for CI projects (having cov_init not null) and set it on NO CI projects

- Median of analysis point (Age on CI adoption)
   - In terms of coverage and build practices
- Process the median for CI projects (those having builds and coverages)
- Set NON-CI projects with the median, all of them.

In [ ]:
medianAge = getMedianCIAdoption()[0][0]
print('Age median for CI adoption: {} days.'.format(medianAge))

print('Setting this median as analysis_point for all NON-CI projects.')
setNonCIProjectAnalysis_Point_Days(medianAge)
print('Done.')

repos = getNonCIProjects()
i=0
for repo in repos:  
    project = repo[0]
    created = repo[1]

    if created is not None:
        analysis_point = created + relativedelta(days=medianAge)

        i += 1
        print('{}/{} - {} Project {} - Created: {} - Days: {} - Analysis: {}'.format(i,len(repos),datetime.now().strftime("%H:%M:%S"),project,created,medianAge,analysis_point))

        setProjectAnalysis_Point(project,analysis_point)


##### 1.4. Setting Analysis_init and Analysis_finish

In [42]:
repos = getProjectsMined()
i=0

for repo in repos:    
    project = repo[0]
    first_commit = repo[1]
    
    #Calculate init of the moth and finish of the last month
    #Adjust the finish date to the first hour of the subsequent month
    analysisInit = analysisFinish = analysisBeforeCI = datetime(first_commit.year, first_commit.month, 1, 0,0,0,tzinfo=first_commit.tzinfo)
    analysisFinish = analysisFinish + relativedelta(months=+13)
    analysisBeforeCI = analysisBeforeCI + relativedelta(months=-12)

    
    setAnalysis_init_finish(project,analysisInit,analysisFinish,analysisBeforeCI)
        
    
    i += 1
    print('{}/{} - Project {} - Init: {} - Finish: {}'.format(i,len(repos),project,analysisInit,analysisFinish))
    #time.sleep(.02)
        

1/876 - Project zzzeek/sqlalchemy - Init: 2013-12-01 00:00:00-03:00 - Finish: 2015-01-01 00:00:00-03:00
2/876 - Project zzanehip/The-OldOS-Project - Init: 2022-01-01 00:00:00-03:00 - Finish: 2023-02-01 00:00:00-03:00
3/876 - Project zyprosoft/ZYChat - Init: 2016-02-01 00:00:00-03:00 - Finish: 2017-03-01 00:00:00-03:00
4/876 - Project zuoqing1988/ZQCNN - Init: 2018-12-01 00:00:00-03:00 - Finish: 2020-01-01 00:00:00-03:00
5/876 - Project zuihou/lamp-cloud - Init: 2018-08-01 00:00:00-03:00 - Finish: 2019-09-01 00:00:00-03:00
6/876 - Project zouzg/mybatis-generator-gui - Init: 2016-12-01 00:00:00-03:00 - Finish: 2018-01-01 00:00:00-03:00
7/876 - Project zlt2000/microservices-platform - Init: 2019-10-01 00:00:00-03:00 - Finish: 2020-11-01 00:00:00-03:00
8/876 - Project zixun/CocoaChinaPlus - Init: 2016-05-01 00:00:00-03:00 - Finish: 2017-06-01 00:00:00-03:00
9/876 - Project zipme/RQShineLabel - Init: 2014-12-01 00:00:00-03:00 - Finish: 2016-01-01 00:00:00-03:00
10/876 - Project ZiniuLu/Pyth

90/876 - Project weppos/whois - Init: 2017-07-01 00:00:00-03:00 - Finish: 2018-08-01 00:00:00-03:00
91/876 - Project wenmingvs/AndroidProcess - Init: 2016-08-01 00:00:00-03:00 - Finish: 2017-09-01 00:00:00-03:00
92/876 - Project WenDesi/lihang_book_algorithm - Init: 2017-02-01 00:00:00-03:00 - Finish: 2018-03-01 00:00:00-03:00
93/876 - Project WeiYe-Jing/datax-web - Init: 2020-06-01 00:00:00-03:00 - Finish: 2021-07-01 00:00:00-03:00
94/876 - Project weaveworks/weave - Init: 2015-04-01 00:00:00-03:00 - Finish: 2016-05-01 00:00:00-03:00
95/876 - Project wcandillon/can-it-be-done-in-react-native - Init: 2019-06-01 00:00:00-03:00 - Finish: 2020-07-01 00:00:00-03:00
96/876 - Project wazuh/wazuh - Init: 2016-03-01 00:00:00-03:00 - Finish: 2017-04-01 00:00:00-03:00
97/876 - Project waynezxcv/Gallop - Init: 2016-08-01 00:00:00-03:00 - Finish: 2017-09-01 00:00:00-03:00
98/876 - Project watson-developer-cloud/swift-sdk - Init: 2017-11-01 00:00:00-03:00 - Finish: 2018-12-01 00:00:00-03:00
99/876 

171/876 - Project Swift-AI/Swift-AI - Init: 2016-06-01 00:00:00-03:00 - Finish: 2017-07-01 00:00:00-03:00
172/876 - Project SusionSuc/AdvancedAndroid - Init: 2019-05-01 00:00:00-03:00 - Finish: 2020-06-01 00:00:00-03:00
173/876 - Project surge-synthesizer/surge - Init: 2019-04-01 00:00:00-03:00 - Finish: 2020-05-01 00:00:00-03:00
174/876 - Project SUPERAndroidAnalyzer/super - Init: 2016-06-01 00:00:00-03:00 - Finish: 2017-07-01 00:00:00-03:00
175/876 - Project SunshineBrother/JHBlog - Init: 2019-02-01 00:00:00-03:00 - Finish: 2020-03-01 00:00:00-03:00
176/876 - Project SunLiner/MiaowShow - Init: 2017-02-01 00:00:00-03:00 - Finish: 2018-03-01 00:00:00-03:00
177/876 - Project summernote/summernote - Init: 2015-08-01 00:00:00-03:00 - Finish: 2016-09-01 00:00:00-03:00
178/876 - Project sucese/android-open-source-project-analysis - Init: 2017-09-01 00:00:00-03:00 - Finish: 2018-10-01 00:00:00-03:00
179/876 - Project subethaedit/SubEthaEdit - Init: 2019-05-01 00:00:00-03:00 - Finish: 2020-06

256/876 - Project Reactive-Extensions/RxJS - Init: 2016-08-01 00:00:00-03:00 - Finish: 2017-09-01 00:00:00-03:00
257/876 - Project razerdp/BasePopup - Init: 2016-08-01 00:00:00-03:00 - Finish: 2017-09-01 00:00:00-03:00
258/876 - Project rasbt/mlxtend - Init: 2019-11-01 00:00:00-03:00 - Finish: 2020-12-01 00:00:00-03:00
259/876 - Project rarnu/root-tools - Init: 2012-04-01 00:00:00-03:00 - Finish: 2013-05-01 00:00:00-03:00
260/876 - Project randdusing/cordova-plugin-bluetoothle - Init: 2014-07-01 00:00:00-03:00 - Finish: 2015-08-01 00:00:00-03:00
261/876 - Project Ranchero-Software/NetNewsWire - Init: 2017-12-01 00:00:00-03:00 - Finish: 2019-01-01 00:00:00-03:00
262/876 - Project radianttap/CardPresentationController - Init: 2019-07-01 00:00:00-03:00 - Finish: 2020-08-01 00:00:00-03:00
263/876 - Project ractivejs/ractive - Init: 2016-04-01 00:00:00-03:00 - Finish: 2017-05-01 00:00:00-03:00
264/876 - Project quran/quran.com-api - Init: 2018-10-01 00:00:00-03:00 - Finish: 2019-11-01 00:00

347/876 - Project Newmu/dcgan_code - Init: 2016-06-01 00:00:00-03:00 - Finish: 2017-07-01 00:00:00-03:00
348/876 - Project newbee-ltd/newbee-mall - Init: 2020-04-01 00:00:00-03:00 - Finish: 2021-05-01 00:00:00-03:00
349/876 - Project Netflix/security_monkey - Init: 2018-05-01 00:00:00-03:00 - Finish: 2019-06-01 00:00:00-03:00
350/876 - Project nenosllc/iCloudDocumentSync - Init: 2013-07-01 00:00:00-03:00 - Finish: 2014-08-01 00:00:00-03:00
351/876 - Project nats-io/nats-streaming-server - Init: 2019-08-01 00:00:00-03:00 - Finish: 2020-09-01 00:00:00-03:00
352/876 - Project nats-io/nats-server - Init: 2020-06-01 00:00:00-03:00 - Finish: 2021-07-01 00:00:00-03:00
353/876 - Project NationalSecurityAgency/ghidra - Init: 2019-09-01 00:00:00-03:00 - Finish: 2020-10-01 00:00:00-03:00
354/876 - Project nate-parrott/Flashlight - Init: 2015-06-01 00:00:00-03:00 - Finish: 2016-07-01 00:00:00-03:00
355/876 - Project naru-jpn/View2ViewTransition - Init: 2017-03-01 00:00:00-03:00 - Finish: 2018-04-0

...

519/876 - Project joaquim-verges/Helium - Init: 2018-10-01 00:00:00-03:00 - Finish: 2019-11-01 00:00:00-03:00
520/876 - Project jmartinesp/SpannedGridLayoutManager - Init: 2018-01-01 00:00:00-03:00 - Finish: 2019-02-01 00:00:00-03:00
521/876 - Project JiongXing/PhotoBrowser - Init: 2017-11-01 00:00:00-03:00 - Finish: 2018-12-01 00:00:00-03:00
522/876 - Project jindongwang/transferlearning - Init: 2017-11-01 00:00:00-03:00 - Finish: 2018-12-01 00:00:00-03:00
523/876 - Project jiajunhui/PlayerBase - Init: 2017-10-01 00:00:00-03:00 - Finish: 2018-11-01 00:00:00-03:00
524/876 - Project jhu-ep-coursera/fullstack-course4 - Init: 2016-09-01 00:00:00-03:00 - Finish: 2017-10-01 00:00:00-03:00
525/876 - Project jfeinstein10/SlidingMenu - Init: 2013-01-01 00:00:00-03:00 - Finish: 2014-02-01 00:00:00-03:00
526/876 - Project jessevig/bertviz - Init: 2019-07-01 00:00:00-03:00 - Finish: 2020-08-01 00:00:00-03:00
527/876 - Project jeecgboot/jeecg-boot - Init: 2019-06-01 00:00:00-03:00 - Finish: 2020-0

606/876 - Project gregdurrett/berkeley-entity - Init: 2015-05-01 00:00:00-03:00 - Finish: 2016-06-01 00:00:00-03:00
607/876 - Project gregdurrett/berkeley-doc-summarizer - Init: 2017-01-01 00:00:00-03:00 - Finish: 2018-02-01 00:00:00-03:00
608/876 - Project graphsketcher/GraphSketcher - Init: 2014-08-01 00:00:00-03:00 - Finish: 2015-09-01 00:00:00-03:00
609/876 - Project gotosleep/JASidePanels - Init: 2012-10-01 00:00:00-03:00 - Finish: 2013-11-01 00:00:00-03:00
610/876 - Project google/trax - Init: 2020-04-01 00:00:00-03:00 - Finish: 2021-05-01 00:00:00-03:00
611/876 - Project googleforgames/open-match - Init: 2019-03-01 00:00:00-03:00 - Finish: 2020-04-01 00:00:00-03:00
612/876 - Project googlearchive/ios-swift-chat-example - Init: 2015-03-01 00:00:00-03:00 - Finish: 2016-04-01 00:00:00-03:00
613/876 - Project googlearchive/code-prettify - Init: 2015-11-01 00:00:00-03:00 - Finish: 2016-12-01 00:00:00-03:00
614/876 - Project googlearchive/android-ConstraintLayoutExamples - Init: 2017-

696/876 - Project DKJone/DKWechatHelper - Init: 2019-08-01 00:00:00-03:00 - Finish: 2020-09-01 00:00:00-03:00
697/876 - Project dinuscxj/LoadingDrawable - Init: 2016-11-01 00:00:00-03:00 - Finish: 2017-12-01 00:00:00-03:00
698/876 - Project dingding3w/DHGuidePageHUD - Init: 2017-02-01 00:00:00-03:00 - Finish: 2018-03-01 00:00:00-03:00
699/876 - Project Dimillian/RedditOS - Init: 2021-02-01 00:00:00-03:00 - Finish: 2022-03-01 00:00:00-03:00
700/876 - Project dillidon/alerts-and-pickers - Init: 2018-08-01 00:00:00-03:00 - Finish: 2019-09-01 00:00:00-03:00
701/876 - Project dianping/cat - Init: 2016-08-01 00:00:00-03:00 - Finish: 2017-09-01 00:00:00-03:00
702/876 - Project devMEremenko/XcodeBenchmark - Init: 2021-03-01 00:00:00-03:00 - Finish: 2022-04-01 00:00:00-03:00
703/876 - Project dev-labs-bg/transitioner - Init: 2018-07-01 00:00:00-03:00 - Finish: 2019-08-01 00:00:00-03:00
704/876 - Project dequan1331/HybridPageKit - Init: 2018-10-01 00:00:00-03:00 - Finish: 2019-11-01 00:00:00-03:

769/876 - Project chaxiu/KotlinJetpackInAction - Init: 2021-01-01 00:00:00-03:00 - Finish: 2022-02-01 00:00:00-03:00
770/876 - Project ChavezChen/CWLateralSlide - Init: 2018-02-01 00:00:00-03:00 - Finish: 2019-03-01 00:00:00-03:00
771/876 - Project chat-sdk/chat-sdk-ios - Init: 2017-07-01 00:00:00-03:00 - Finish: 2018-08-01 00:00:00-03:00
772/876 - Project CharlinFeng/CorePhotoBroswerVC - Init: 2015-12-01 00:00:00-03:00 - Finish: 2017-01-01 00:00:00-03:00
773/876 - Project CharlesPikachu/Games - Init: 2019-01-01 00:00:00-03:00 - Finish: 2020-02-01 00:00:00-03:00
774/876 - Project chainer/chainer - Init: 2019-10-01 00:00:00-03:00 - Finish: 2020-11-01 00:00:00-03:00
775/876 - Project celzero/rethink-app - Init: 2021-01-01 00:00:00-03:00 - Finish: 2022-02-01 00:00:00-03:00
776/876 - Project Ccixyj/JBusDriver - Init: 2017-11-01 00:00:00-03:00 - Finish: 2018-12-01 00:00:00-03:00
777/876 - Project catarse/catarse - Init: 2013-04-01 00:00:00-03:00 - Finish: 2014-05-01 00:00:00-03:00
778/876 -

856/876 - Project Alecrim/AlecrimCoreData - Init: 2015-03-01 00:00:00-03:00 - Finish: 2016-04-01 00:00:00-03:00
857/876 - Project alanwangmodify/WXSTransition - Init: 2017-01-01 00:00:00-03:00 - Finish: 2018-02-01 00:00:00-03:00
858/876 - Project akshay2211/BubbleTabBar - Init: 2020-03-01 00:00:00-03:00 - Finish: 2021-04-01 00:00:00-03:00
859/876 - Project ajstarks/svgo - Init: 2010-10-01 00:00:00-03:00 - Finish: 2011-11-01 00:00:00-03:00
860/876 - Project ajaxorg/ace - Init: 2019-08-01 00:00:00-03:00 - Finish: 2020-09-01 00:00:00-03:00
861/876 - Project AiYinZiLeGong/PUBG-Radar - Init: 2018-08-01 00:00:00-03:00 - Finish: 2019-09-01 00:00:00-03:00
862/876 - Project airbnb/visx - Init: 2020-09-01 00:00:00-03:00 - Finish: 2021-10-01 00:00:00-03:00
863/876 - Project airbnb/react-dates - Init: 2019-09-01 00:00:00-03:00 - Finish: 2020-10-01 00:00:00-03:00
864/876 - Project aidlearning/AidLearning-FrameWork - Init: 2019-11-01 00:00:00-03:00 - Finish: 2020-12-01 00:00:00-03:00
865/876 - Proje

##### 1.5. (Re)Process metrics quantities in analysis period:
- Notebook 3_1:
    - analysis_releases
    - analysis_issues
    - analysis_prs
- Notebook 5_1:
    - Mine Communication of pull requests in selected releases
- Notebook 6:
    - Consolidate pullrequests data via Github API
- Notebook 7:
    - qty_bugs_period
    - qty_bugs
    - Merge_conflicts

## Metrics for CI classification
* Build Health (Median)
    * Rate of builds passed.
* Builds activity (Median)
    * Frequency of days having builds in a given period of time (e.g. 10/30 days having builds in a month).
* Time to Fix broken builds (Median)
    * Time since a build fail status until the next success.
* Test Coverage (Median)
    * Percent of test coverage in a build

### Process
- Get analysis period (projects.analysis_init - projects.analysis_finish) for each project:
    - Calculate Build Health, Build activity (integration frequency), Time to Fix broken builds, and Test Coverage;
    - Get projects having values for coverage and build greater than 0;

#### Calculate and record metric fields on table Projects

In [ ]:
repos = getProjects()

for repo in repos:    
    project = repo[0]
    analysisInit = repo[1]
    analysisFinish = repo[2]
    print('Processing CI Metrics. Project:\t {}\t {}  -  {}'.format(project,analysisInit,analysisFinish))
    
    buildHealth = calcBuildHealth(project,analysisInit,analysisFinish)
    timeToFix = calcTimeToFix(project,analysisInit,analysisFinish)
    builds_activity = calcActivity(project,analysisInit,analysisFinish)
    coverage = calcCoverage(project,analysisInit,analysisFinish)
    
    updateProjectMetrics(project,buildHealth, timeToFix, builds_activity,coverage)

#### Flag CI Projects
- Projects having:
    - coverage > 0;
    - Build (activity, health, ttf) > 0;
    - prs_selected and issues_selected
    - Travis CI project
    
#### Select CI Projects
- Projects having:
    - CI flag true (previous bullet);
    - Bugs in analysis period;
    
#### Draw the same quantity of projects through NO CI instances
- Projects having:
    - No CI server
    - prs_selected and issues_selected
    - Bugs in analysis period


In [ ]:
updateCIProjects()

qty_projects = getQtyProjects()

# Which of these groups is the lower?
# Select all projects from this lower group
# Then, draw the same quantity from the other.
if qty_projects[0][1] < qty_projects[1][1]:
    setSelectedAllProjects(qty_projects[0][0])
    drawProjects(qty_projects[1][0],qty_projects[0][1])
else:
    setSelectedAllProjects(qty_projects[1][0])
    drawProjects(qty_projects[0][0],qty_projects[1][1])
    
#drawNOCIProjects(1)

In [ ]:
def drawProjects(ci,qty):
    projects = getGroupProjects(ci)
    projects = list(map(lambda x: x[0], projects))
    i=0
    
    while i < qty:
        random.shuffle(projects)
        id = random.randint(0, len(projects)-1)
        proj = projects.pop(id)
        
        i +=1
        
        print('PROJECT: {}'.format(proj))
        try:
            query = """UPDATE  projects
                set rq1_included = true
                WHERE repo_name like %s"""

            connection = connectDB()
            cursor = connection.cursor()
            cursor.execute(query,[proj])
            connection.commit()
            cursor.close()
            connection.close()
        except psycopg2.IntegrityError as e:
            print ("==============================================================")
            print ("Error while updating into PostgreSQL. drawProjects >>> Exception: {}".format(e)) 
            connection.close()
        except Exception as e:
            print ("==============================================================")
            print ("Error while processing drawProjects >>> Exception: {}".format(e)) 

In [ ]:
def getGroupProjects(ci):
    query = """SELECT repo_name From projects 
                WHERE analysis_issues > 0 AND analysis_prs > 0 AND commits_mined is true 
                AND analysis_releases > 0 AND qty_bugs > 0"""
    
    if ci:
        query += """ AND builds_activity > 0 AND coverage > 0 AND ci_service is not null;"""
    else:
        query += """ AND ci_service is NULL;"""
            

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [ ]:
def setSelectedAllProjects(ci):
    try:
        query = """UPDATE  projects
            set rq1_included = true
            WHERE analysis_issues > 0 AND analysis_prs > 0 AND commits_mined is true 
                    AND analysis_releases > 0 AND qty_bugs > 0"""
        
        if ci:
            query += """ AND builds_activity > 0 AND coverage > 0 AND ci_service is not null;"""
        else:
            query += """ AND ci_service is null;"""
            

        connection = connectDB()
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
        cursor.close()
        connection.close()
    except psycopg2.IntegrityError as e:
        print ("==============================================================")
        print ("Error while updating into PostgreSQL. setSelectedAllProjects >>> Exception: {}".format(e)) 
        connection.close()
    except Exception as e:
        print ("==============================================================")
        print ("Error while processing setSelectedAllProjects >>> Exception: {}".format(e)) 

In [ ]:
def getQtyProjects():
    query = """select ci,count(repo_name) from projects WHERE 
                    analysis_issues > 0 AND analysis_prs > 0 AND commits_mined is true AND analysis_releases > 0 AND qty_bugs > 0 AND 
                    ((builds_activity > 0 AND coverage > 0 AND ci_service is not null) 
                    OR ci_service is null)
                group by ci;"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [ ]:
def updateCIProjects():
    try:
        query = """UPDATE  projects 
            set ci = true
            WHERE repo_name IN
                (select repo_name from projects WHERE 
                    (commits_mined IS TRUE and analysis_releases > 0 and analysis_issues > 0 and analysis_prs > 0 
                    and qty_bugs > 0
                    and analysis_point_days is not null)
                AND CI_SERVICE iLIKE 'TRAVIS CI'
                AND coverage > 0 AND builds_activity > 0);                
            """

        connection = connectDB()
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
        cursor.close()
        connection.close()
    except psycopg2.IntegrityError as e:
        print ("==============================================================")
        print ("Error while updating into PostgreSQL. updateCIProjects >>> Exception: {}".format(e)) 
        connection.close()
    except Exception as e:
        print ("==============================================================")
        print ("Error while processing updateCIProjects >>> Exception: {}".format(e)) 

In [ ]:
def drawNOCIProjects(qty):
    projects = getNOCIProjects()
    projects = list(map(lambda x: x[0], projects))
    i=0
    
    while i < qty:
        random.shuffle(projects)
        id = random.randint(0, len(projects)-1)
        proj = projects.pop(id)
        
        i +=1
        
        print('PROJECT: {}'.format(proj))
        try:
            query = """UPDATE  projects
                set rq1 = true
                WHERE repo_name like %s"""

            connection = connectDB()
            cursor = connection.cursor()
            cursor.execute(query,[proj])
            connection.commit()
            cursor.close()
            connection.close()
        except psycopg2.IntegrityError as e:
            print ("==============================================================")
            print ("Error while updating into PostgreSQL. drawNOCIProjects >>> Exception: {}".format(e)) 
            connection.close()
        except Exception as e:
            print ("==============================================================")
            print ("Error while processing drawNOCIProjects >>> Exception: {}".format(e)) 

In [ ]:
#calcActivity('3b1b/manim','2019-10-01 00:00:00-03:00','2020-10-01 00:00:00-03:00')
#r=getBuildDays('3b1b/manim','2019-10-01 00:00:00-03','2020-10-01 00:00:00-03')
#len(r)

#r=getCoverage('ruslanskorb/RSKImageCropper','2015-03-01 00:00:00-03:00','2016-03-01 00:00:00-03:00')
#type(r)
#if r.empty:
#    print('eita')
#calcCoverage('3b1b/manim','2019-10-01 00:00:00-03','2020-10-01 00:00:00-03')
#r
#print(r['coverage'].median())
#r = removeOutliers(r)
#print(r['coverage'].median())
#failBuilds = getFailBuilds('3b1b/manim')
#nextDate = getNextSuccess('3b1b/manim',failBuilds[0][1])
#nextDate[0][1]
#diff = (nextDate[0][1] - failBuilds[0][1]).total_seconds()
#diff
#fillTimeToNextSuccess('3b1b/manim')

#calcTimeToFix('3b1b/manim','2019-10-01 00:00:00-03','2020-10-01 00:00:00-03')
#ttfs = getTTFs('3b1b/manim','2019-10-01 00:00:00-03','2020-10-01 00:00:00-03')
#ttfs['time_to_fix'].median()

### Aux Functions

In [2]:
import psycopg2
import os
import requests 
import time
import pytz    
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import dateutil.parser
import random
import pandas as pd
from scipy import stats
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

In [3]:
def connectDB():
    f = open('/home/psql_pwd.txt', "r")
    pwd = f.readline().replace('\n','')
    
    return psycopg2.connect(user = "ci_quality",
                              password = pwd,
                              host = "127.0.0.1",
                              port = "5432",
                              database = "Causal_CI_Quality_v4")

In [4]:
def connectDBPandas():
    f = open('/home/psql_pwd.txt', "r")
    pwd = f.readline().replace('\n','')
    
    #"User ID=ci_quality;Password={};Host=localhost;Port=5432;Database=Causal_CI_Quality_v4;Pooling=true;Min Pool Size=0;Max Pool Size=100;Connection Lifetime=0;".format(pwd)
    DATABASE_URI = 'postgresql://ci_quality:{}@localhost:5432/Causal_CI_Quality_v4'.format(pwd)
    #connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
    
    return create_engine(DATABASE_URI)

In [5]:
def getProjects():
    #query = """SELECT repo_name, min(init_period), max(end_period) From metrics_period 
    #    WHERE period ilike 'month' and repo_name IN 
    #        (SELECT repo_name From PROJECTS WHERE (prs_selected IS TRUE and issues_selected is true) AND CI_SERVICE iLIKE 'TRAVIS CI')
    #    GROUP BY repo_name
    #    order by repo_name --offset 400"""

    query = """SELECT repo_name, analysis_init, analysis_finish, analysis_point From PROJECTS 
    WHERE commits_mined IS TRUE and analysis_releases > 0 and analysis_issues > 0 and analysis_prs > 0 
    and qty_bugs > 0
    and analysis_point_days is not null
    and CI_SERVICE ILIKE 'TRAVIS CI'
    order by repo_name;"""
    
    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [6]:
def getProjectsToRecalculate():
    query = """SELECT repo_name, created From PROJECTS 
    WHERE CI_SERVICE ILIKE 'TRAVIS CI'
    order by repo_name;"""
    
    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [7]:
def getNOCIProjects():
    query = """SELECT repo_name From PROJECTS 
    WHERE commits_mined2 IS TRUE and analysis_releases > 1 and analysis_issues > 0 and analysis_prs > 0 
    and qty_bugs_period > 0
    and CI_SERVICE is NULL
    order by repo_name;"""
    

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [8]:
def getAllProjects():
    query = """SELECT repo_name From PROJECTS 
    WHERE commits_mined2 IS TRUE and analysis_releases > 1 and analysis_issues > 0 and analysis_prs > 0 
    and qty_bugs_period > 0
    order by repo_name;"""
    
    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [9]:
def calcBuildHealth(project,init, finish):
    builds = getBuildResults(project, init, finish)
    buildHealth = computeBuildHealth(builds)

    print('\t\t\t Build Health: {} - Total builds: {}'.format(buildHealth,len(builds)))
    return buildHealth

In [10]:
def calcTimeToFix(project,init, finish):
    fillTimeToNextSuccess(project)
    
    ttfs = getTTFs(project, init, finish)
    if ttfs.empty:
        ttf = 0
    else:
        ttf = ttfs['time_to_fix'].median()
        
    print('\t\t\t Time To Fix: {} - Qty data ttf: {}'.format(ttf,ttfs['time_to_fix'].count()))
    return ttf

In [11]:
def fillTimeToNextSuccess(project):
    #Get all failed builds
    failBuilds = getFailBuilds(project)
    #For each build, get the next success and update it.
    for b in failBuilds:
        fail = b[1]
        nextDate = getNextSuccess(project,fail)#created
        if nextDate is not None and len(nextDate) > 0:
            success = nextDate[0][1]
        
            diff = (success - fail).total_seconds()
        
            updateBuildTTF(project,b[0],diff)

In [12]:
def calcActivity(project,init, finish):
    #init = dateutil.parser.parse(init)
    #finish = dateutil.parser.parse(finish)
    days = (finish - init).days
    
    build_days = getBuildDays(project, init, finish)
    build_activity = len(build_days)/days

    print('\t\t\t Build Activity: {} - Total Activity Days: {} - Total Days: {}'.format(build_activity,len(build_days),days))
    return build_activity

In [13]:
def calcCoverage(project,init, finish):
    covs = getCoverage(project, init, finish)
    if covs.empty:
        coverage = 0
    else:
        if checkValues(covs['coverage']):
            coverage = covs['coverage'].median()
        else:
           # coverage = removeOutliers(covs)
            coverage = covs['coverage'].median()
        
        #coverage = covs['coverage'].median()
        
    print('\t\t\t Coverage: {} - Qty data cov: {}'.format(coverage,covs['coverage'].count()))
    return coverage

In [14]:
def getBuildResults(repo_name,initDate, finishDate):    
    query = """SELECT result FROM builds_mined WHERE 
            repo_name = %s AND ((STARTED_at BETWEEN %s AND %s) OR (finishED_at BETWEEN %s AND %s))"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name,initDate, finishDate,initDate, finishDate])
    rows = cursor.fetchall()
    
    
    cursor.close()
    connection.close()
    return rows

In [15]:
def getFailBuilds(repo_name):    
    query = """select build_number, started_at, result from builds_mined 
        WHERE repo_name = %s AND result is False AND started_at is not NULL AND time_to_fix is NULL
        order by started_at"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchall()
    
    
    cursor.close()
    connection.close()
    return rows

In [16]:
def getNextSuccess(repo_name,date):    
    query = """select build_number, started_at, result from builds_mined 
        WHERE repo_name = %s AND result is True  AND started_at > %s
        order by started_at limit 1;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name,date])
    rows = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return rows

In [17]:
def getBuildDays(project, init, finish):
    query = """select distinct date(started_at) from builds_mined
                WHERE repo_name = %s AND (STARTED_at BETWEEN %s AND %s);"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [project,init, finish])
    rows = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return rows

In [18]:
def getCoverage(project_name,init,finish):
    connection = connectDBPandas()
    query = 'select coverage from coverage where (created BETWEEN %s AND %s) AND repo_name like %s;'

    df = pd.read_sql_query(query,con=connection,params=[init,finish,project_name])
    #connection.close()
    
    #Check if column contains all values 0. In this case we do not filter outliers.
    #if checkValues(df['coverage']):
    return df
    #else:
    #    return removeOutliers(df)

In [19]:
def getTTFs(project_name,init,finish):
    connection = connectDBPandas()
    query = 'select time_to_fix from builds_mined where time_to_fix is not Null AND repo_name like %s AND (started_at BETWEEN %s AND %s);'

    df = pd.read_sql_query(query,con=connection,params=[project_name,init,finish])
    #connection.close()
    #Check if column contains all values 0. In this case we do not filter outliers.
    #if checkValues(df['time_to_fix']):
    #    return df
    #else:
    #    return removeOutliers(df)
    return df

In [20]:
def computeBuildHealth(builds):
    results = list(map(lambda x: x[0], builds))
    success = list(filter(lambda x: x == True, results))
    
    if len(results) > 0:
        return len(success)/len(results)
    
    return 0

In [21]:
def updateProjectMetrics(repo_name,build_health, timeToFix, builds_actvity,coverage):
    try:
        query = """UPDATE  projects 
            set build_health = %s,
                time_to_fix_broken_builds = %s,
                builds_activity = %s,
                coverage = %s
            WHERE repo_name like %s"""

        connection = connectDB()
        cursor = connection.cursor()
        cursor.execute(query, [build_health, timeToFix, builds_actvity,coverage,repo_name])
        connection.commit()
        cursor.close()
        connection.close()
    except psycopg2.IntegrityError as e:
        print ("==============================================================")
        print ("Error while updating into PostgreSQL. updateProjectMetrics >>> Exception: {}".format(e)) 
        print('Project: {}'.format(repo_name))
        connection.close()
    except Exception as e:
        print ("==============================================================")
        print ("Error while processing updateProjectMetrics >>> Exception: {}".format(e)) 
        print('Project: {}'.format(repo_name))

In [22]:
def updateBuildTTF(project, build_number, timeToFix):
    try:
        query = """UPDATE  builds_mined 
            set time_to_fix = %s
            WHERE repo_name like %s AND build_number = %s"""

        connection = connectDB()
        cursor = connection.cursor()
        cursor.execute(query, [timeToFix, project, build_number])
        connection.commit()
        cursor.close()
        connection.close()
    except psycopg2.IntegrityError as e:
        print ("==============================================================")
        print ("Error while updating into PostgreSQL. updateBuildTTF >>> Exception: {}".format(e)) 
        print('Project: {}    PR - {} '.format(repo_name, pr_number))
        connection.close()
    except Exception as e:
        print ("==============================================================")
        print ("Error while processing updateBuildTTF >>> Exception: {}".format(e)) 
        print('Project: {}    PR - {}  '.format(repo_name, pr_number))

In [23]:
#https://www.kite.com/python/answers/how-to-remove-outliers-from-a-pandas-dataframe-in-python
def removeOutliers(df):
    z_scores = stats.zscore(df)
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 3).all(axis=1)
    df_filtered = df[filtered_entries]

    return df_filtered

In [24]:
def checkValues(df):
    if (df == 0).all():
        return True
    else:
        i=v=0
        v = df[0]
        for a in df:
            if a != v:
                return True
            
        return False

In [25]:
def countPRsProject(project, mergeConflict=None):
    connection = connectDB()
    cursor = connection.cursor()
    
    if mergeConflict is None or mergeConflict is False:
        query = """SELECT count(id) from pullrequests WHERE project_name like %s"""
    else:
        query = """SELECT count(id) from pullrequests WHERE project_name like %s and mergeconflict is True"""

    cursor.execute(query, [project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [26]:
def countIssuesProject(project, bug=None):
    connection = connectDB()
    cursor = connection.cursor()
    
    if bug is None or bug is False:
        query = """SELECT count(id) from issue WHERE repo_name like %s"""
    elif bug is True:
        query = """SELECT count(id) from issue WHERE repo_name like %s and isbug is True"""

    cursor.execute(query, [project])
    row = cursor.fetchone()
    
    cursor.close()
    connection.close()
    if row is not None:
        return row[0]
    else:
        return None

In [27]:
def updateProjectQty(repo_name,qtd_issues, qtd_pull_requests, qtd_bugs,qtd_merge_conflics):
    try:
        query = """UPDATE  projects 
            set qtd_issues = %s,
                qtd_pull_requests = %s,
                qtd_bugs = %s,
                qtd_merge_conflics = %s
            WHERE repo_name like %s"""

        connection = connectDB()
        cursor = connection.cursor()
        cursor.execute(query, [qtd_issues, qtd_pull_requests, qtd_bugs,qtd_merge_conflics,repo_name])
        connection.commit()
        cursor.close()
        connection.close()
    except psycopg2.IntegrityError as e:
        print ("==============================================================")
        print ("Error while updating into PostgreSQL. updateProjectQty >>> Exception: {}".format(e)) 
        print('Project: {}'.format(repo_name))
        connection.close()
    except Exception as e:
        print ("==============================================================")
        print ("Error while processing updateProjectQty >>> Exception: {}".format(e)) 
        print('Project: {}'.format(repo_name))

In [28]:
def getFirstCov(repo_name):    
    query = """SELECT min(created) FROM coverage WHERE 
            repo_name = %s;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchone()
    
    cursor.close()
    connection.close()
    return rows

In [29]:
def getFirstBuild(repo_name):    
    query = """SELECT min(started_at) FROM builds_mined WHERE 
            repo_name = %s;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchone()
    
    cursor.close()
    connection.close()
    return rows

In [30]:
def setCovInit(repo_name,cov_init,cov_init_days):
    try:
        query = """UPDATE  projects
            set cov_init = %s,
                cov_init_days = %s
            WHERE repo_name like %s;"""
            

        connection = connectDB()
        cursor = connection.cursor()
        cursor.execute(query,[cov_init,cov_init_days,repo_name])
        connection.commit()
        cursor.close()
        connection.close()
    except psycopg2.IntegrityError as e:
        print ("==============================================================")
        print ("Error while updating into PostgreSQL. setCovInit >>> Exception: {}".format(e)) 
        connection.close()
    except Exception as e:
        print ("==============================================================")
        print ("Error while processing setCovInit >>> Exception: {}".format(e)) 

In [31]:
def getNonCIProjects():
    query = """select repo_name, created from projects
            where qtd_stars >= 100 and forked is false and size  > 10000
            and ci_service is null 
            order by repo_name;"""
    
    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [32]:
def setProjectAnalysis_Point(repo_name,date):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """UPDATE  projects 
    set analysis_point = %s
    WHERE repo_name like %s;"""

    cursor.execute(query, [date,repo_name])
        
    connection.commit()
    cursor.close()
    connection.close()

In [33]:
def getProjectsMined():
    query = """SELECT repo_name, analysis_point From PROJECTS 
    where  qtd_stars >= 100 and forked is false and size  > 10000
            and ((ci_service is null or ci_service like 'Travis CI') and analysis_point is not null)
            order by repo_name desc;"""

    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [34]:
def setAnalysis_init_finish(repo_name,date_init,date_finish,analysisBeforeCI):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """UPDATE  projects 
    set analysis_init = %s,
    analysis_finish = %s,
    analysis_before_ci = %s
    WHERE repo_name like %s;"""

    cursor.execute(query, [date_init,date_finish,analysisBeforeCI,repo_name])
        
    connection.commit()
    cursor.close()
    connection.close()

In [35]:
def getMedianCIAdoption():
    query = """select  PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY ci_service_init_days) from projects
            where analysis_issues > 0 and analysis_prs > 0 and commits_mined is true AND analysis_releases > 0
                AND qty_bugs > 0 
                AND analysis_point_days is not null
                AND builds_activity > 0
                AND coverage > 0
                and ci_service ilike 'Travis CI';"""
    #AND builds_activity > 0
    #AND coverage > 0
    
    connection = connectDB()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    connection.close()
    return result

In [36]:
def setNonCIProjectAnalysis_Point_Days(days):
    connection = connectDB()
    cursor = connection.cursor()
    
    query = """UPDATE  projects 
    set analysis_point_days = %s
    WHERE qtd_stars >= 100 
        and forked is false 
        and size  > 10000
        and ci_service is null;"""

    cursor.execute(query, [days])
        
    connection.commit()
    cursor.close()
    connection.close()

In [37]:
def getCIProjectsMetrics(ci=None):
    
    if ci is None:
        query = """select qtd_stars, qtd_issues, qtd_issues_in_period,qtd_pull_requests,qtd_pull_request_in_period,qtd_bugs,qtd_merge_conflicts,COVERAGE,BUILDS_ACTIVITY, TIME_TO_FIX_BROKEN_BUILDS, BUILD_HEALTH
                from projects where rq1 is true;"""
    elif ci is True:
        query = """select qtd_stars, qtd_issues, qtd_issues_in_period,qtd_pull_requests,qtd_pull_request_in_period,qtd_bugs,qtd_merge_conflicts,COVERAGE,BUILDS_ACTIVITY, TIME_TO_FIX_BROKEN_BUILDS, BUILD_HEALTH
                from projects where rq1 is true AND ci is True;"""
    else:
        query = """select qtd_stars, qtd_issues, qtd_issues_in_period,qtd_pull_requests,qtd_pull_request_in_period,qtd_bugs,qtd_merge_conflicts,COVERAGE,BUILDS_ACTIVITY, TIME_TO_FIX_BROKEN_BUILDS, BUILD_HEALTH
                from projects where rq1 is true AND ci is false;"""
    
    connection = connectDB()
    df = pd.read_sql_query(query,con=connection)
    connection.close()
    
    return df

In [38]:
def getBuildDates(repo_name):    
    query = """SELECT min(STARTED_at),max(STARTED_at) FROM builds_mined WHERE 
            repo_name = %s;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return rows

In [39]:
def getCovsDates(repo_name):    
    query = """SELECT min(created),max(created) FROM coverage WHERE 
            repo_name = %s;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return rows

In [40]:
def getBugsDates(repo_name):    
    query = """SELECT min(created_at),max(created_at) FROM issue 
            WHERE repo_name = %s AND isbug is true;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return rows

In [41]:
def getReleasesDates(repo_name):    
    query = """SELECT min(created_at),max(created_at) FROM RELEASES 
            WHERE repo_name = %s AND isbug is true;"""

    connection = connectDB()
    cursor = connection.cursor()
    
    cursor.execute(query, [repo_name])
    rows = cursor.fetchall()
    
    cursor.close()
    connection.close()
    return rows

In [ ]:
'''projects = getAllProjects()

i=1
for proj in projects:
    project = proj[0]
    
    print('Updating project {}/{}  --  {}'.format(i,len(projects),project))
    total_prs = countPRsProject(project)
    total_merge_conflicts = countPRsProject(project,True)
    total_issues = countIssuesProject(project)
    total_bugs = countIssuesProject(project,True)
    
    updateProjectQty(project,total_issues, total_prs, total_bugs,total_merge_conflicts)
    i+=1
    
    dates_covs = getCovsDates(project)
    dates_bugs = getBugsDates(project)
    dates_releases = getReleasesDates(project)
'''

In [ ]:
#projects = getCIProjectsMetrics(ci=None)
#projects.sum()